## Pull in UDFs

In [ ]:
%run nb_udfs


## Get Tenant Settings

In [ ]:
workspace = 'Admin%20Center' #have to escape the & symbol and spaces
lakehouse = 'lh_monitoring'

tenant_settings = 'dimTenantSettings'


In [ ]:
#get a list of all tenant and load to a table in the lakehouse
response = _base_api(
        request=f"/v1/admin/tenantsettings",
        method="get"
    )
df_tenant = pd.json_normalize(response.json()['value'])
columns_with_arrays = ["enabledSecurityGroups","excludedSecurityGroups","properties"]
columns_with_arrays = [acol for acol in columns_with_arrays if acol in df_tenant.columns]


In [ ]:
#loops over columns and converts the array columns to strings so we don't break the delta table load
for acol in columns_with_arrays:
    if acol in df_tenant.columns:
        df_tenant[acol] = df_tenant[acol].apply(str)
df_tenant = spark.createDataFrame(df_tenant)

#df_tenant.printSchema()
#print(df_tenant.columns)
#df_tenant.show(1)

In [ ]:
#creates a slowly changing dimension so we can keep an eye on any deleted tenant
tenantPath = udf_GetFilePath(workspace, lakehouse, tenant_settings)
#print(tenantPath)
naturalKeyColumnList = ['title']
primaryKeyColumnName = "tableId"

returnVal = udf_UpsertDimension(df_tenant,2,tenantPath,naturalKeyColumnList,primaryKeyColumnName,True)
print(returnVal)